In [2]:
import pandas as pd
import numpy as np

# import tensorflow_decision_forests as tfdf

In [3]:
!pip install tensorflow tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 33.8 MB/s eta 0:00:00


In [4]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
!pwd

/content


In [7]:
!ls /content/drive

MyDrive


In [8]:
!ls /content/drive/MyDrive/Assignment_1/Data-RF.xlsx

/content/drive/MyDrive/Assignment_1/Data-RF.xlsx


In [9]:
df = pd.read_excel("/content/drive/MyDrive/Assignment_1/Data-RF.xlsx")

In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Lab-Test1(30)     500 non-null    float64
 1   Lab-Test2(24)     500 non-null    int64  
 2   Midsem Test (90)  500 non-null    float64
 3   Gender            500 non-null    object 
 4   Attendance        500 non-null    object 
 5   Grade             500 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 23.6+ KB


In [11]:
df.head()

,Lab-Test1(30),Lab-Test2(24),Midsem Test (90),Gender,Attendance,Grade
0,13.00,24,66.0,Male,High,A
1,15.00,24,67.0,Female,High,A
2,5.25,24,45.0,Male,High,B-
3,2.75,19,34.0,Male,High,C-
4,7.25,24,30.0,Male,High,C-


In [12]:
df["Gender"].nunique()

4

In [13]:
unique_values = df["Gender"].unique()
print(unique_values)

['Male' 'Female' 'Female ' 'Male ']


In [14]:
df["Gender"] = df["Gender"].replace({'Male ':'Male','Female ':'Female'})

In [15]:
df["Gender"].nunique()

2

In [16]:
uniq = df["Attendance"].unique()
print(uniq)

['High' 'Low' 'Moderate']


In [17]:
Att_Map = {'High':3,'Low':2,'Moderate':1}
df["Attendance"] = df['Attendance'].replace(Att_Map)

In [18]:
grade_map = {"A":10,"A-":9,"B":8,"B-":7,"C":6,"C-":5,"D":4,"E":2}
df['Grade'] = df['Grade'].replace(grade_map)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Lab-Test1(30)     500 non-null    float64
 1   Lab-Test2(24)     500 non-null    int64  
 2   Midsem Test (90)  500 non-null    float64
 3   Gender            500 non-null    object 
 4   Attendance        500 non-null    int64  
 5   Grade             500 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 23.6+ KB


In [20]:
df['Gender'] = df['Gender'].replace({'Male':1,'Female':0})

In [21]:
df.head()

,Lab-Test1(30),Lab-Test2(24),Midsem Test (90),Gender,Attendance,Grade
0,13.00,24,66.0,1,3,10
1,15.00,24,67.0,0,3,10
2,5.25,24,45.0,1,3,7
3,2.75,19,34.0,1,3,5
4,7.25,24,30.0,1,3,5


In [37]:
X = df.drop(['Grade'],axis = 1)
y = df['Grade']

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

In [24]:
df_train = pd.concat([X_train,y_train],axis = 1)
df_test = pd.concat([X_test,y_test],axis = 1)

In [25]:
df_train.shape


(350, 6)

In [32]:
df_test.head()

,Lab-Test1(30),Lab-Test2(24),Midsem Test (90),Gender,Attendance,Grade
361,6.25,24,55.0,1,3,8
73,12.00,24,53.0,0,3,8
374,13.25,24,68.0,0,1,10
155,12.50,20,0.0,1,3,4
104,19.00,24,60.0,1,3,10


In [42]:
ds_train = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label="Grade")
ds_test = tfdf.keras.pd_dataframe_to_tf_dataset(df_test,label = "Grade")
# test_dataset_size = tf.data.experimental.cardinality(ds_train).numpy()
# print("Size of test_dataset:", test_dataset_size)
# print(df_test.shape)

Size of test_dataset: 1
(150, 6)


In [28]:
model = tfdf.keras.RandomForestModel()
model.fit(ds_train)


Use /tmp/tmpyhkvnd7l as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.126087. Found 350 examples.
Training model...
Model trained in 0:00:00.273125
Compiling model...
Model compiled.


In [29]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (5):
	Attendance
	Gender
	Lab-Test1(30)
	Lab-Test2(24)
	Midsem_Test_(90)

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "Midsem_Test_(90)"  0.807314 ################
    2.    "Lab-Test1(30)"  0.364165 #####
    3.    "Lab-Test2(24)"  0.164206 
    4.       "Attendance"  0.157200 
    5.           "Gender"  0.157014 

Variable Importance: NUM_AS_ROOT:
    1. "Midsem_Test_(90)" 240.000000 ################
    2.    "Lab-Test1(30)" 60.000000 

Variable Importance: NUM_NODES:
    1. "Midsem_Test_(90)" 4557.000000 ################
    2.    "Lab-Test1(30)" 4198.000000 ##############
 

In [48]:
model.compile(metrics = ["accuracy"])

evaluation = model.evaluate(ds_test,return_dict = True)
print()
for name,value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 171ms/step - loss: 0.0000e+00 - accuracy: 0.8667

loss: 0.0000
accuracy: 0.8667


**This is the first tree of our random forest model**

In [67]:
tfdf.model_plotter.plot_model_in_colab(model,tree_idx = 0)